In [63]:

import os

import pandas as pd
import numpy as np


In [64]:
#plotting libraries and setting defaults

%matplotlib inline

import seaborn as sns
from seaborn import plt

# Some nice default configuration for plots
plt.rcParams['figure.figsize'] = 10, 7.5
plt.rcParams['axes.grid'] = True

In [108]:
offers_df = pd.read_csv('/Users/rychughes/GA/AVS-Kaggle-Data/offers.gz', 
                        compression='gzip') 
print offers_df.shape
print offers_df.head(3)

(37, 6)
     offer  category  quantity    company  offervalue  brand
0  1190530      9115         1  108500080        5.00  93904
1  1194044      9909         1  107127979        1.00   6732
2  1197502      3203         1  106414464        0.75  13474


In [109]:
train_hist_df = pd.read_csv('/Users/rychughes/GA/AVS-Kaggle-Data/trainHistory.gz', 
                        compression='gzip') 
print train_hist_df.shape
print train_hist_df.head(3)

(160057, 7)
         id  chain    offer  market  repeattrips repeater   offerdate
0     86246    205  1208251      34            5        t  2013-04-24
1     86252    205  1197502      34           16        t  2013-03-27
2  12682470     18  1197502      11            0        f  2013-03-28


In [110]:
#merge
train_hist_offers_df = pd.merge( train_hist_df, offers_df, how='inner')

print train_hist_offers_df.shape
train_hist_offers_df.head(5)

(160057, 12)


,id,chain,offer,market,repeattrips,repeater,offerdate,category,quantity,company,offervalue,brand
0,86246,205,1208251,34,5,t,2013-04-24,2202,1,104460040,2,3718
1,15753725,17,1208251,4,0,f,2013-04-24,2202,1,104460040,2,3718
2,16535563,4,1208251,1,7,t,2013-04-27,2202,1,104460040,2,3718
3,18259179,3,1208251,2,0,f,2013-04-24,2202,1,104460040,2,3718
4,21024070,15,1208251,9,1,t,2013-04-23,2202,1,104460040,2,3718


In [111]:
%%bash
gunzip /Users/rychughes/GA/AVS-Kaggle-Data/transactions.csv.gz -c | wc -l

Process is terminated.


In [117]:
col_df = pd.read_csv( '/Users/rychughes/GA/AVS-Kaggle-Data/transactions.csv.gz', compression='gzip', nrows=1)
col_headers = list( col_df.columns.values)
col_headers

['id',
 'chain',
 'dept',
 'category',
 'company',
 'brand',
 'date',
 'productsize',
 'productmeasure',
 'purchasequantity',
 'purchaseamount']

In [114]:
#create empty df to append too
train_df = pd.DataFrame()

In [128]:
from __future__ import division

col_df = pd.read_csv( '/Users/rychughes/GA/AVS-Kaggle-Data/transactions.csv.gz', compression='gzip', nrows=1)
col_headers = list( col_df.columns)
print col_headers

n=0
chunk_size = 1000000
rows_hist_train = 5000000
#rows_hist_train = 349655790 #actual figure, from bash cmd

#iterate through transactions
for skip_rows in range( 0, rows_hist_train, chunk_size):
    temp_df = pd.read_csv('/Users/rychughes/GA/AVS-Kaggle-Data/transactions.csv.gz', 
                          compression= 'gzip', 
                          nrows= chunk_size, 
                          #header= col_headers, 
                          skiprows= skip_rows)
    
    #test
    print n
    n = n+1
    
    #output
    temp_df.columns = col_headers
    temp_df_2 = pd.merge( temp_df, train_hist_offers_df, how='inner')
    train_df = train_df.append( temp_df_2)

['id', 'chain', 'dept', 'category', 'company', 'brand', 'date', 'productsize', 'productmeasure', 'purchasequantity', 'purchaseamount']
0
1
2
3
4


In [129]:
print train_df.shape
train_df.head(4)

(599, 18)


,id,chain,dept,category,company,brand,date,productsize,productmeasure,purchasequantity,purchaseamount,offer,market,repeattrips,repeater,offerdate,quantity,offervalue
0,86252,205,32,3203,106414464,13474,2012-06-07,5.00,OZ,1,2.99,1197502,34,16,t,2013-03-27,1,0.75
1,86252,205,32,3203,106414464,13474,2013-03-03,5.00,OZ,1,1.99,1197502,34,16,t,2013-03-27,1,0.75
2,13807224,4,56,5616,104610040,15889,2012-08-01,6.67,OZ,1,3.00,1204576,1,0,f,2013-04-05,1,1.00
3,13807224,4,56,5616,104610040,15889,2012-08-01,7.50,OZ,1,3.00,1204576,1,0,f,2013-04-05,1,1.00


In [99]:
from __future__ import division

col_df = pd.read_csv( '/Users/rychughes/GA/AVS-Kaggle-Data/transactions.csv.gz', compression='gzip', nrows=1)
col_headers = list( col_df.columns)
#col_headers

n=0
chunk_size = 1000000
rows_hist_train = 5000000
#rows_hist_train = 349655790 #actual figure, from bash cmd
filtered_dfs = []
# for skip_rows in range( 0, rows_hist_train, chunk_size):
#     print chunk_size, skip_rows
for temp_df in pd.read_csv('/Users/rychughes/GA/AVS-Kaggle-Data/transactions.csv.gz', 
                          compression='gzip',
                         chunksize=chunk_size):
    #output
    df = pd.merge( temp_df, train_hist_offers_df, how='inner')
    if df is not None:
        filtered_dfs.append(df)
    
train_df = pd.concat(filtered_dfs)

KeyboardInterrupt: 

In [98]:
#view train df
print train_df.shape

(160057, 12)
